In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cosmoHammer as cm
import keras as ks

Using TensorFlow backend.


In [2]:
%ls

 98bk02.h5         cov_inv_test       index.png     params_test
 binary_data_02/   dTb02              lvac02.png   'paramter estimation.ipynb'
 bk_norm02         EmuBk0.2.h5        nbins02       script.py
 Bk_test           hyperparam.ipynb   N_bins_test   x_HI02
 bk_un02           index              params02


In [3]:
N_bins_test = np.loadtxt('N_bins_test')
N_bins_test.shape

(18, 550)

In [4]:
Bk_test = np.loadtxt('Bk_test')
Bk_test.shape

(18, 550)

In [6]:
params_test = np.loadtxt('params_test')
params_test.shape

(18, 3)

In [7]:
def cov(Bk,Nbins):
    a = np.eye(550,550)
    b = Bk/np.sqrt(Nbins)
    b = np.reshape(b,(550,1))
    b = b*a
    cov_inv = np.linalg.inv(b)
    return cov_inv

In [8]:
def Bk_model(params):
    
    model = ks.models.load_model('98bk02.h5')

    Bk_model = model.predict([[params]])
    Bk_model = Bk_model*100.
    
    return Bk_model[0]

In [9]:
def lL(params,Bk,N_bins):
    if params[0]<=0 or params[0] > 200 or params[1]<=0 or params[1] > 150 or params[2]<=0 or params[2] > 1500 :
        l=-np.inf
    else:

        model=Bk_model(params)
        DMU=Bk-model
        DMU = np.reshape(DMU, (1,550))
        cov_inv = cov(Bk,N_bins)
        
        l=-0.5*np.dot(np.dot(DMU,cov_inv),DMU.T)
        l = l[0][0]
   
    return l

In [10]:
ndim=3            #===== no. of parameters =======#

nwalkers=6      #========= no. of random walkers ========#

Nion = np.random.randint(20,200,6).T
Rmfp = np.random.randint(20,120,6).T
NoH = np.random.randint(50,1100,6).T
p = np.zeros(shape=(6,3))
p[:,0] = Nion
p[:,1] = Rmfp
p[:,2] = NoH


In [11]:
lL(p[0],Bk_test[0],N_bins_test[0])

-919132.9334914201

In [12]:
import emcee
import os

os.environ["OMP_NUM_THREADS"] = "1"

nsteps = 3000

sampler=emcee.EnsembleSampler(nwalkers,ndim,lL,args=[Bk_test[0],N_bins_test[0]])


In [ ]:
#start = time.time()
sampler.run_mcmc(p, nsteps, progress=True)
#end = time.time()
#multi_time = end - start
#print("Multiprocessing took {0:.1f} seconds".format(multi_time))



In [ ]:
with open("script.py", "w") as f:
    f.write("""
import sys
import time
import emcee
import numpy as np
from schwimmbad import MPIPool
import keras as ks


def Bk_model(params):
    
    model = ks.models.load_model('98bk02.h5')

    Bk_model = model.predict([[params]])
    Bk_model = Bk_model*100.
    
    return Bk_model[0]

def cov(Bk,Nbins):
    a = np.eye(550,550)
    b = Bk/np.sqrt(Nbins)
    b = np.reshape(b,(550,1))
    b = b*a
    cov_inv = np.linalg.inv(b)
    return cov_inv
    
    
def lL(params,Bk,N_bins):
    if params[0]<=0 or params[0] > 200 or params[1]<=0 or params[1] > 150 or params[2]<=0 or params[2] > 1500 :
        l=-np.inf
    else:

        model=Bk_model(params)
        DMU=Bk-model
        DMU = np.reshape(DMU, (1,550))
        cov_inv = cov(Bk,N_bins)
        
        l=-0.5*np.dot(np.dot(DMU,cov_inv),DMU.T)
        l = l[0][0]
   
    return l
    
    
with MPIPool() as pool:
    if not pool.is_master():
        pool.wait()
        sys.exit(0)

    np.random.seed(42)
    ndim=3            #===== no. of parameters =======#

    nwalkers=6      #========= no. of random walkers ========#

    Nion = np.random.randint(20,200,6).T
    Rmfp = np.random.randint(20,120,6).T
    NoH = np.random.randint(50,1100,6).T
    p = np.zeros(shape=(6,3))
    p[:,0] = Nion
    p[:,1] = Rmfp
    p[:,2] = NoH
    nsteps = 100
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, pool=pool)
    start = time.time()
    sampler.run_mcmc(p, nsteps)
    end = time.time()
    print(end - start)
""")

mpi_time = !mpiexec -n {ncpu} python script.py
mpi_time = float(mpi_time[0])
print("MPI took {0:.1f} seconds".format(mpi_time))
print("{0:.1f} times faster than serial".format(serial_time / mpi_time))


In [ ]:
%ls
